In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input, Flatten, GlobalMaxPool1D, SpatialDropout1D

from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors # Gensim contains word2vec models and processing tools
from gensim.scripts.glove2word2vec import glove2word2vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import binarize

from nltk import word_tokenize

import re
import os
import nltk


import random
random.seed(0)
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
tf.__version__

#!{sys.executable} -m pip install pypdf2
#!{sys.executable} -m pip install python-docx
#!{sys.executable} -m pip install python-docx2txt

Using TensorFlow backend.


'2.0.0'

In [2]:
#!{sys.executable} -m pip install docx2txt
#!{sys.executable} -m pip install textract
#!{sys.executable} -m pip install antiword

In [3]:
import docx
import docx2txt

In [4]:
import win32com.client
word = win32com.client.Dispatch("Word.Application")
word.visible = False

In [5]:
import PyPDF2

In [6]:
import textract

In [7]:
filepath = 'C:\\Resumes'
entries = os.listdir(filepath)

In [8]:
cvs = pd.DataFrame(entries, columns = ["name"])

In [9]:
cvs

,name
0,Abhishek Gandhi_Technical Business Analyst_CA.doc
1,Abishek Gopal_React JS Developer_WI.doc
2,Adam Altieri_Business Analyst_AZ.doc
3,Ahmad Qasem - PM.docx
4,Ahmad Qasem_Project Manager_IL.doc
5,Alexander Abalakov_Azure Architect_CANADA.doc
6,Amit Misra.doc
7,Andrew Dolopo_Virtualization Engineer _CA.doc
8,Apurva Keshoju_React JS Developer_MD.doc
9,Arti K -Sr. Recruiter.docx


In [10]:
def readtxt(filename):
    #return textract.process(filename)
    print (filename)
    wb = word.Documents.Open(filename)
    doc = word.ActiveDocument
    fullText = []
    
    for para in doc.paragraphs:
        fullText.append(str(para).replace("\r","\n"))
    word.Documents.Close()
    return "".join(fullText)

In [11]:
#readtxt("C:\Abhishek_Gandhi_Technical_Business_Analyst_CA.doc")
#"C:\\2018.txt"
#readtxt()

In [12]:
def updateFileText(filename):
    return readtxt(filepath + "\\" + filename)

In [13]:
cvs_range = len(cvs)
cvs["text"] = cvs["name"].apply(updateFileText)

C:\Resumes\Abhishek Gandhi_Technical Business Analyst_CA.doc
C:\Resumes\Abishek Gopal_React JS Developer_WI.doc
C:\Resumes\Adam Altieri_Business Analyst_AZ.doc
C:\Resumes\Ahmad Qasem - PM.docx
C:\Resumes\Ahmad Qasem_Project Manager_IL.doc
C:\Resumes\Alexander Abalakov_Azure Architect_CANADA.doc
C:\Resumes\Amit Misra.doc
C:\Resumes\Andrew Dolopo_Virtualization Engineer _CA.doc
C:\Resumes\Apurva Keshoju_React JS Developer_MD.doc
C:\Resumes\Arti K -Sr. Recruiter.docx
C:\Resumes\Arulnathan Savarimuthu_Infrastructure Architect_OH.doc
C:\Resumes\avinashkumarsingh_profile.docx
C:\Resumes\Beverly Arukwe_Scrum Master_CA.doc
C:\Resumes\Charanjit Singh Channi_Oracle Financial Functinal_TX.doc
C:\Resumes\Chiranjeevi Gajjeli_Azure Arcchitect_GA.doc
C:\Resumes\Dave Svab - CSEO profile.pdf
C:\Resumes\Dean L. Carrera_Infrastructure Architect_GA.doc
C:\Resumes\Deepak Jasapara_Oracle Financial Functional_CA.doc
C:\Resumes\FARAZ KHAN CV.docx
C:\Resumes\Gary Rogensues_Infrastructure Engineer_MI.doc
C:\Res

In [14]:
cvs

,name,text
0,Abhishek Gandhi_Technical Business Analyst_CA.doc,"Abhishek Gandhi\nSacramento, CA\n316-734-7049\..."
1,Abishek Gopal_React JS Developer_WI.doc,Abishek Gopal\nMadison WI\n408 753 7832\nreach...
2,Adam Altieri_Business Analyst_AZ.doc,"Adam Altieri\nPhoenix, AZ\n303-578-5319\nAdama..."
3,Ahmad Qasem - PM.docx,"Ahmad Qasem\nChicago, IL\n312-723-2889\nahmad...."
4,Ahmad Qasem_Project Manager_IL.doc,"Ahmad Qasem\nChicago, IL\n312-723-2889\nahmad...."
5,Alexander Abalakov_Azure Architect_CANADA.doc,"Alexander Abalakov\nVancouver, CA\n(778) 829-8..."
6,Amit Misra.doc,Amit Misra\n(SAP S4 HANA Certified SD CONSULTA...
7,Andrew Dolopo_Virtualization Engineer _CA.doc,"Andrew Dolopo\nMission Viejo, CA\n(949) 547-48..."
8,Apurva Keshoju_React JS Developer_MD.doc,"Apurva Keshoju\nBaltimore, MD\n410-302-8366\na..."
9,Arti K -Sr. Recruiter.docx,Arti Khanuja\nartikhanuja@outlook.com 425-246-...


In [15]:
cvs_category = pd.read_csv("C:\CVs.csv")

In [16]:
df = pd.concat([cvs,cvs_category], join = "inner", axis = 1)

In [17]:
df.head()

,name,text,name,Cloud Arch,BA,Developer,Manager,Consultant
0,Abhishek Gandhi_Technical Business Analyst_CA.doc,"Abhishek Gandhi\nSacramento, CA\n316-734-7049\...",Abhishek Gandhi_Technical Business Analyst_CA.doc,0,1,0,0,1
1,Abishek Gopal_React JS Developer_WI.doc,Abishek Gopal\nMadison WI\n408 753 7832\nreach...,Abishek Gopal_React JS Developer_WI.doc,0,0,1,0,0
2,Adam Altieri_Business Analyst_AZ.doc,"Adam Altieri\nPhoenix, AZ\n303-578-5319\nAdama...",Adam Altieri_Business Analyst_AZ.doc,0,1,0,0,1
3,Ahmad Qasem - PM.docx,"Ahmad Qasem\nChicago, IL\n312-723-2889\nahmad....",Ahmad Qasem - PM.docx,0,0,0,1,0
4,Ahmad Qasem_Project Manager_IL.doc,"Ahmad Qasem\nChicago, IL\n312-723-2889\nahmad....",Ahmad Qasem_Project Manager_IL.doc,0,0,0,1,0


In [18]:
df = df.drop(["name"], axis = 1)

In [19]:
df

,text,Cloud Arch,BA,Developer,Manager,Consultant
0,"Abhishek Gandhi\nSacramento, CA\n316-734-7049\...",0,1,0,0,1
1,Abishek Gopal\nMadison WI\n408 753 7832\nreach...,0,0,1,0,0
2,"Adam Altieri\nPhoenix, AZ\n303-578-5319\nAdama...",0,1,0,0,1
3,"Ahmad Qasem\nChicago, IL\n312-723-2889\nahmad....",0,0,0,1,0
4,"Ahmad Qasem\nChicago, IL\n312-723-2889\nahmad....",0,0,0,1,0
5,"Alexander Abalakov\nVancouver, CA\n(778) 829-8...",1,0,0,0,0
6,Amit Misra\n(SAP S4 HANA Certified SD CONSULTA...,0,0,0,0,1
7,"Andrew Dolopo\nMission Viejo, CA\n(949) 547-48...",0,0,1,0,0
8,"Apurva Keshoju\nBaltimore, MD\n410-302-8366\na...",0,0,1,0,0
9,Arti Khanuja\nartikhanuja@outlook.com 425-246-...,0,0,0,0,0


In [20]:
max_features = 10000
MAX_LENGTH = 1000
embedding_size = 200

def update_headline(headline):
    headline_new = headline.lower()
    headline_new = re.sub('[^a-zA-z0-9\s]','',headline_new)
    headline_new = headline_new.replace('rt',' ')
    return headline_new

df["text"] = df["text"].apply(update_headline)
df.head()

,text,Cloud Arch,BA,Developer,Manager,Consultant
0,abhishek gandhi\nsacramento ca\n3167347049\nof...,0,1,0,0,1
1,abishek gopal\nmadison wi\n408 753 7832\nreach...,0,0,1,0,0
2,adam altieri\nphoenix az\n3035785319\nadamalta...,0,1,0,0,1
3,ahmad qasem\nchicago il\n3127232889\nahmadelsh...,0,0,0,1,0
4,ahmad qasem\nchicago il\n3127232889\nahmadelsh...,0,0,0,1,0


In [21]:
x = df["text"]

In [22]:
vocabSize = max_features
tokenizer = Tokenizer(num_words=vocabSize, split=' ')
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)
x = pad_sequences(x,MAX_LENGTH)

In [23]:
x

array([[   3, 2704, 3745, ...,    1,   10,  633],
       [3874, 3875,  324, ...,  814,    1,  345],
       [ 101,  692,  487, ...,    6,  116,  457],
       ...,
       [   0,    0,    0, ..., 1998,   18,   43],
       [   0,    0,    0, ...,    1, 1130,  734],
       [  26,  358,   11, ..., 1748,    9,  254]])

In [24]:
y = df["Consultant"]

In [25]:
y = np.array(y)

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [27]:
y_train

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1], dtype=int64)

In [28]:
y_test

array([0, 0, 0, 0, 1, 0], dtype=int64)

In [29]:
#### Add your code here ####
VOCAB_SIZE = 10000
MAX_LENGTH = 1000

model = Sequential() #creating sequential model
model.add(Embedding(input_dim = VOCAB_SIZE, output_dim = 300, input_length = MAX_LENGTH)) #adding embedding layer
model.add(SpatialDropout1D(0.4)) #added for robustness
model.add(LSTM(units = 300, return_sequences = True, recurrent_dropout = 0.1,)) #adding LSTM layer
#model.add(TimeDistributed(Dense(100, activation = "relu"))) # timedistributed layer
model.add(Flatten())# Flatten 
model.add(Dense(1,activation="sigmoid")) #Dense layer

In [30]:
#### Add your code here ####
model.compile(optimizer= "adam", loss ="binary_crossentropy",metrics =["accuracy"])

In [31]:
class_weights = {0: 2,
                1: 3}

In [32]:
fitted_model = model.fit(x_train, y_train, batch_size = 64, epochs=5, verbose=1, class_weight = class_weights)

Train on 24 samples
Epoch 1/5
24/24 [==============================] - 12s 514ms/sample - loss: 1.6428 - accuracy: 0.5000
Epoch 2/5
24/24 [==============================] - 8s 343ms/sample - loss: 1.1754 - accuracy: 0.9583
Epoch 3/5
24/24 [==============================] - 11s 459ms/sample - loss: 0.8302 - accuracy: 1.0000
Epoch 4/5
24/24 [==============================] - 10s 414ms/sample - loss: 0.5839 - accuracy: 1.0000
Epoch 5/5
24/24 [==============================] - 10s 426ms/sample - loss: 0.3149 - accuracy: 1.0000


In [33]:
score,acc = model.evaluate(x_test, y_test,verbose = 1)

6/1 [====================================================================================================================================================================================] - 1s 198ms/sample - loss: 0.4129 - accuracy: 1.0000


In [34]:
y_pred = model.predict(x_test)

In [35]:
from sklearn.preprocessing import binarize
y_pred_new = binarize(y_pred, 0.5)

In [36]:
y_pred_new = [item for sublist in y_pred_new for item in sublist]

In [37]:
print(classification_report(y_test,y_pred_new))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         1

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6



In [38]:
consultant_dict = dict({1:"Consultant",0:"Not Consultant"})
SAMPLE_INDEX = 1
p = model.predict(np.array([x_test[SAMPLE_INDEX]]))
if p > 0.5:
    print("Consultant")
else:
    print("Not Consultant")

print(consultant_dict.get(y_test[SAMPLE_INDEX]))
print(x_test[SAMPLE_INDEX])

Not Consultant
Not Consultant
[  58 1683  143    1 1037 1996   10 1468  231  138   10  130   64  160
 2116   81   30 5153 1469 1038 1685  472 5154 5155    1  575  294  575
  248 5156  120  116   96 2451 2381  695  120  165 5157 5158    1  305
   78  804  537 2745  666  252 1911 5159  409  850   84  128 1620 3258
 1541  435  151   38   27   45 3121 2976   69 5160  609 1015 3259   10
 1468 1071   11  859 5161   57 5162    7   15 5163    4   10  126   18
   66   11  389 1667   60  198  126 2824 2223  654    1 5164   10 1468
 1055 5165  116  954    1  782   11 1068   11 1686    1   50   45 5166
  389  120  165  120  358 1997   18  853 1998   18   43   81  157  456
  408   10   28  130  408 5167 5168 3258 3260   88  946   34  293 1236
   34   60   59 1848 3169   81  131  253 1531   32    1 5169 1687 1105
 1807   31    1 1678  149 1014    1 1047 1678   17  115    9   14  119
  295 1381    9   14  124 2952  145  580  234  310    1  531    3 1080
    9  607    8  234  256  381  145    2  298  

In [39]:
import math

In [45]:
math.degrees(math.atan2(-1,0))

-90.0